In [1]:
# %%capture
# !pip install qiskit qiskit-aer qiskit-ibm-runtime qiskit_experiments pylatexenc quantuminspire snntorch qiskit tqdm matplotlib

In [2]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [3]:
# @title Imports
# from qiskit import QuantumCircuit, Aersimu, execute
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import os
import numpy as np
import pandas as pd
import torch

from src.Quantum_circuits import *
from src.QSVAE_model import *
from src.POVM_dataset import *
# from torchvision import transforms, utils
import matplotlib.gridspec as gridspec

if torch.cuda.is_available():
 dev = "cuda:0"
#  dev = "cpu"
else:
 dev = "cpu"
device = torch.device(dev)
print(device)
torch.__version__

# J_ROOT = os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID'])
# print(J_ROOT)

cuda:0


'2.4.1+cu121'

In [4]:
# @title Main (Global definitions)
n = 3 # Amount of qubits
shots = 10e1 # amount of shots taken by the quantum simulator
first_run = True
# Support for "Starmon-5" and "AerSimulator"
backend_type = "AerSimulator"
backend = select_backend(backend_type)
train = True
test = True
val = True

# Define hyperparameters
beta = 0.819
num_steps = 200
num_epochs = 1
learning_rate = 1e-3
batch_train, batch_test, batch_val = (10000, 200, 1000)
num_workers = 0
shuffle = False
split = [0.6, 0.2]

In [5]:
quantum_exp = QuantumExperiment(backend, n, shots)
quantum_exp.run_experiment()
# quantum_exp.print_circuits_with_counts()
train_loader, test_loader, val_loader = load_data(quantum_exp,
                                                  first_run,
                                                  backend,
                                                  n,
                                                  shots,
                                                  split,
                                                  [batch_train, batch_test, batch_val],
                                                  shuffle, num_workers)

# Instantiate the model for the given parameters
model = SQVAE(n=n, batch_size=[batch_train, batch_train, batch_val],
              beta=beta, num_steps=num_steps, learning_rate=learning_rate, device=device)

# Run the model and get the fidelity for the current parameter setting
fidelity_score = model.run(train=True, test=False, val=True,
                           train_loader=train_loader, test_loader=test_loader,
                           val_loader=val_loader, num_epochs=num_epochs)


Dataset and circuit saved.


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1, Loss: 272.9964


  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([20, 16])
The fidelity for 3 qubits is 0.9629821457879684 with 1000 samples.


In [6]:
# Reproduction of paper
parameters = [
    (3, 100, 20_000),
    (4, 200, 100_000),
    (5, 500, 4**5 * 500),  # 4^5 * 500
    (6, 600, 4**6 * 500),  # 4^6 * 500
    (7, 800, 4**7 * 500),  # 4^7 * 500
    (8, 1000, 4**8 * 500)  # 4^8 * 500
]

# Run the model for each parameter setting and calculate fidelity:
fidelities = []
for param in parameters:
    n, batch_train, batch_val = param  # Unpack parameters
    quantum_exp = QuantumExperiment(backend, n, shots)
    quantum_exp.run_experiment()
#     quantum_exp.print_circuits_with_counts()
    train_loader, test_loader, val_loader = load_data(quantum_exp,
                                                      first_run,
                                                      backend,
                                                      n,
                                                      shots,
                                                      split,
                                                      [batch_train, batch_test, batch_val],
                                                      shuffle, num_workers)

    # Instantiate the model for the given parameters
    model = SQVAE(n=n, batch_size=[batch_train, batch_train, batch_val],
                beta=beta, num_steps=num_steps, learning_rate=learning_rate, device=device)

    # Run the model and get the fidelity for the current parameter setting
    fidelity_score = model.run(train=True, test=False, val=True,
                            train_loader=train_loader, test_loader=test_loader,
                            val_loader=val_loader, num_epochs=num_epochs)
    # Append the fidelity score to the list
    fidelities.append(fidelity_score)

# Plot the histogram of fidelities
plot_histogram(fidelities, parameters)

Dataset and circuit saved.


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1, Loss: 407.9185


  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([20, 16])
The fidelity for 3 qubits is 0.8875487981083946 with 20000 samples.
Dataset and circuit saved.


...

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1, Loss: 220.8575


  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([20, 512])


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.53 GiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 3.17 GiB is allocated by PyTorch, and 982.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)